# 주차어때 크롤링


## 1. 주차장 정보에서 url 가져오기

### 1-1 csv 불러오기, 검색어 정제

In [2]:
import pandas as pd
import re             # 정규표현식

In [3]:
data = pd.read_csv("서울시 공영주차장 안내 정보.csv", encoding='utf-8')

In [4]:
columns = ['주차장코드','주차장명', '주소'] #, '주차장 위치 좌표 위도', '주차장 위치 좌표 경도']
data2 = data[columns].copy()
data2.head(3)

,주차장코드,주차장명,주소
0,1010089,초안산근린공원주차장(구),도봉구 창동 24-0
1,1012254,마들스타디움(근린공원)(구),노원구 상계동 770-2
2,1013181,마장동(건물) 공영주차장(구),성동구 마장동 463-2


In [17]:
data['주차장코드'].value_counts()

1538796    310
1503518    299
1495387    244
1552996    194
1536637    175
          ... 
1579833      1
1579832      1
1579598      1
1579566      1
984688       1
Name: 주차장코드, Length: 1107, dtype: int64

데이터에 중복된 column이 존재한다. 중복된 주차장코드의 column들이 존재하는 이유는 다음과 같다.

> 앞 부분만 보시면 동일한 데이터로 보일 수 있으나, 시트 가장 오른쪽의 위도/경도 컬럼을 확인해 보시면 값이 다르게 표시되고 있습니다.
> 노상주차장의 경우 도로의 한쪽을 주차구역으로 만든 것으로, 주차장을 설치한 도로 길이만큼 위도/경도값이 여럿일 수 밖에 없습니다.
> 위와 같은 이유로 시작점이나 중간점, 끝점을 하나 지정하셔서 대표값으로 사용하시면 될 것으로 보입니다.
> -서울시 공영주차장 안내 정보 관리자

리뷰는 그룹화하여 제공되므로, 그룹 별로 시작값만 남기고 제거한다. 중복이 사라져 많이 비므로 인덱스를 재설정한다. 중복 제거 전 16000개의 데이터가 있었는데 제거 후 1107개가 남는다.

In [10]:
data2 = data2.drop_duplicates(['주차장코드'])
data2 = data2.reset_index(drop=True)
data2[50:55]

,주차장코드,주차장명,주소
50,1235916,청계2(남) 공영주차장(시),중구 입정동 29-1
51,1236307,청계1(남) 공영주차장(시),중구 수표동 65-4
52,1236612,남대문 화물 공영주차장(시),중구 남대문로4가 24-1
53,1236742,중앙일보사옆 관광버스전용 주차장(시),중구 순화동 7-0
54,1236949,한진면세점앞 노상 공영주차장(구),중구 남대문로5가 533-0 0


주차장명을 그대로 검색하면 주차장이 안 나오는 경우가 있다.
다음과 같은 원칙에 따라 주차장명을 가공해 검색어를 새로 만든다.
* 괄호 안에 들어가는 말은 지운다.
* '주차장' 이라는 단어가 포함되어 있지 않은 경우 끝에 붙인다.

예시: '은평평화공원(구)' ---> '은평평화공원 주차장'

In [6]:
def rename_parking_lot(old_name):
    new_name = re.sub(r'\([^)]*\)', '', old_name) # 괄호 안에 든 문자열 제거
    if '주차장' not in new_name:
        new_name = new_name + " 주차장"          # 주차장 이라는 문자열 없으면 붙여줌 
    return new_name

In [19]:
data2['검색어'] = data2['주차장명'].map(rename_parking_lot)
data2.head(5)

,주차장코드,주차장명,주소,검색어
0,1010089,초안산근린공원주차장(구),도봉구 창동 24-0,초안산근린공원주차장
1,1012254,마들스타디움(근린공원)(구),노원구 상계동 770-2,마들스타디움 주차장
2,1013181,마장동(건물) 공영주차장(구),성동구 마장동 463-2,마장동 공영주차장
3,1025695,영등포여고 공영(구),영등포구 신길동 184-3,영등포여고 공영 주차장
4,1025696,당산근린공원 공영(구),영등포구 당산동3가 385-0,당산근린공원 공영 주차장


### 1-2 URL 저장
URL 이라는 이름의 열을 만들어, '검색어' 열에 대한 검색 결과를 보여주는 URL을 저장한다.

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import time

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [58]:
data2['url'] = ''
data2.head(2)

,주차장코드,주차장명,주소,검색어,url
0,1010089,초안산근린공원주차장(구),도봉구 창동 24-0,초안산근린공원주차장,
1,1012254,마들스타디움(근린공원)(구),노원구 상계동 770-2,마들스타디움 주차장,


url 저장 전에 해당 열을 공백으로 초기화해준다.

In [ ]:
for i, keyword in enumerate(data2['검색어'].tolist()):
    try:
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'
        # 검색 url
        driver.get(naver_map_search_url)
        time.sleep(4) # 대기
        cu = driver.current_url
        res_code = re.findall(r"place/(\d+)", cu) # 번호 알아냄
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'
        data2['url'][i] = final_url
    except IndexError:
        data2['url'][i] = ''
        
data2

In [60]:
data2.to_csv('url_completed_in_progress.csv', encoding = 'utf-8-sig')

완성한 데이터프레임을 csv 파일로 저장한다. 

## 2. URL 별 리뷰 크롤링

1. 에서 추출한 URL을 바탕으로 크롤링을 진행한다.

### 2-1 유효한 URL만 추출 

In [61]:
import pandas as pd
import time
import re
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import InvalidArgumentException

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [62]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

driver = webdriver.Chrome(service=Service('chromedriver.exe'), options=chrome_options)



In [78]:
df = pd.read_csv('url_completed_in_progress.csv')

In [79]:
df['rank'] = 0

주차장 하나에 해당하는 리뷰들을 하나의 csv 파일로 저장한다.

평균별점 같은 경우에 csv 파일에 덧붙인다.

In [ ]:
for i in range(df.shape[0]):
        print("===========================")
        print(str(i)+'번째 주차장')
        try:
            driver.get(df['url'][i])
            thisurl = df['url'][i]
            time.sleep(2)
        except InvalidArgumentException:
            print("-url 없음-")
            continue
        
        # 더보기 버튼 다 누르기 
        while True:
            try:
                time.sleep(1)
                driver.find_element(by=By.TAG_NAME, value='body').send_keys(Keys.END)
                time.sleep(3)
                driver.find_element(by=By.CSS_SELECTOR, value='#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.cXO6M > div._2kAri > a').click()
                time.sleep(3)
                driver.find_element(by=By.TAG_NAME, value='body').send_keys(Keys.END)
                time.sleep(1)
            except NoSuchElementException:
                print("-더보기버튼 모두 클릭 완료-")
                break
        # 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        try:
            rank = soup.select('#app-root > div > div > div > div.place_section.GCwOh > div._3uUKd._2z4r0 > div._37n49 > span._1Y6hi._1A8_M > em')[0].text
            df['rank'][i] = int(rank)    # 평균 별점
        except:
            pass
        
        comments = []
        for comment in soup.find_all('span', class_='WoYOw'):
            comments.append(comment.get_text())    # 리뷰들 
        
        # csv 파일로 저장
        # 경로 = "reviews/주차장코드.csv"
        df2 = pd.DataFrame({"Comments":comments})
        file_name = "reviews/" + str(df['주차장코드'][i]) + ".csv"
        df2.to_csv(file_name, index= False, encoding='utf-8') #파일경로
        

In [122]:
driver.quit()    # 크롤링 종료

## 3. 리뷰 데이터 전처리 

### 3-1 데이터 정제하기

정규 표현식을 사용하여, 한글을 제외하고 모두 제거해준다.

데이터가 제일 많은 1441353.csv에 대해 선행 진행

In [115]:
#import할 패키지 목록
import re
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
#from pykospacing import Spacing
from hanspell import spell_checker
from collections import Counter

In [95]:
review = pd.read_csv("reviews/1441353.csv", encoding='utf-8')
review.head(10)

,Comments
0,굿
1,좋아요
2,협소ㅜㅜ주말은헬
3,좋아요
4,주말엔\n차 많아서 대기 해야해요
5,주차장 정보
6,굿이여 ~~~~~~
7,좋아요
8,가로수길 갈때 자주 애용합니다.
9,굿


In [97]:
def extract_word(text):
    hangul = re.compile('[^가-힣]')
    result = hangul.sub(' ', text)
    return result

In [98]:
review['Comments'] = review['Comments'].apply(lambda x:extract_word(x))

형태소 분석을 위해 맞춤법을 교정한다.

In [110]:
def spell_check(text):
    spelled_sent = spell_checker.check(text)
    hanspell_sent = spelled_sent.checked
    return hanspell_sent

In [111]:
review['Comments'] = review['Comments'].apply(lambda x:spell_check(x))

형태소 분석은 Okt 모델을 이용한다.

In [ ]:
okt = Okt()
words = " ".join(review['Comments'].tolist())
words = okt.morphs(words, stem=True)
words 

불용어를 제거한다.

In [119]:
with open('stopwords.txt', 'r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")
remove_stopwords = [x for x in words if x not in stopwords]
remove_stopwords

['굿',
 '좋다',
 '협소',
 '주말',
 '은',
 '헬',
 '좋다',
 '주말',
 '엔',
 '차',
 '많다',
 '대기',
 '해',
 '주차장',
 '정보',
 '굿',
 '이여',
 '좋다',
 '가로수길',
 '갈다',
 '자주',
 '애용',
 '굿',
 '굿',
 '굿',
 '굿',
 '편리하다',
 '굿',
 '주차',
 '편하다',
 '혼잡',
 '좋다',
 '좋다',
 '아유',
 '좋다',
 '좋다',
 '굿',
 '맘',
 '곳',
 '좋다',
 '좋다',
 '좋다',
 '위치',
 '좋다',
 '좋다',
 '굿굿',
 '굿',
 '신사',
 '내',
 '살다',
 '저렴하다',
 '편이',
 '라',
 '차갑다',
 '많다',
 '가로수길',
 '접근성',
 '도',
 '좋다',
 '도보',
 '분',
 '내외',
 '다니다',
 '수',
 '좋다',
 '굿',
 '좋다',
 '굿',
 '굿',
 '굿',
 '좋다',
 '분당',
 '원',
 '위치',
 '상',
 '저렴하다',
 '편',
 '가로수길',
 '접근성',
 '좋다',
 '공영',
 '주차장',
 '가격',
 '도',
 '저렴하다',
 '좋다',
 '굿',
 '좋다',
 '좋다',
 '굿',
 '좋다',
 '가로수길',
 '골목',
 '들어오다',
 '주차',
 '걸리다',
 '집',
 '가깝다',
 '거리',
 '괜히',
 '차',
 '가져오다',
 '대기',
 '버리다',
 '토요일',
 '주말',
 '엔',
 '웬만하다',
 '차보',
 '다',
 '대중교통',
 '도보',
 '거',
 '귯',
 '주차',
 '줄',
 '짱',
 '임',
 '좋다',
 '좋다',
 '굿',
 '굿굿',
 '좋다',
 '굿',
 '넓다',
 '좋다',
 '굿',
 '좋다',
 '굿',
 '좋다',
 '굿',
 '편하다',
 '가로수길',
 '공영',
 '주차장',
 '좋다',
 '좋다',
 '좋다',
 '좋다',
 '굿',
 '괜찮다',
 '좋다',
 '좋다',
 '분',

In [121]:
frequent = Counter(remove_stopwords).most_common()
frequent

[('좋다', 90),
 ('굿', 43),
 ('주차', 33),
 ('주차장', 30),
 ('가로수길', 26),
 ('편하다', 13),
 ('저렴하다', 12),
 ('공영', 12),
 ('갈다', 9),
 ('비싸다', 8),
 ('은', 7),
 ('위치', 7),
 ('원', 7),
 ('주말', 6),
 ('곳', 6),
 ('도', 6),
 ('이용', 6),
 ('공', 6),
 ('편리하다', 5),
 ('분', 5),
 ('수', 5),
 ('할인', 5),
 ('대기', 4),
 ('가격', 4),
 ('넓다', 4),
 ('강남', 4),
 ('한', 4),
 ('는', 4),
 ('후', 4),
 ('엔', 3),
 ('차', 3),
 ('많다', 3),
 ('굿굿', 3),
 ('신사', 3),
 ('편', 3),
 ('걸리다', 3),
 ('괜찮다', 3),
 ('싸다', 3),
 ('쌈', 3),
 ('게', 3),
 ('방문', 3),
 ('공해', 3),
 ('자주', 2),
 ('차갑다', 2),
 ('접근성', 2),
 ('도보', 2),
 ('거리', 2),
 ('다', 2),
 ('거', 2),
 ('줄', 2),
 ('만', 2),
 ('듯', 2),
 ('그나마', 2),
 ('최고', 2),
 ('인', 2),
 ('일요일', 2),
 ('기준', 2),
 ('주변', 2),
 ('마음', 2),
 ('동네', 2),
 ('차량', 2),
 ('추천', 2),
 ('경차', 2),
 ('프로', 2),
 ('신구', 2),
 ('초교', 2),
 ('압구정', 2),
 ('가로수', 2),
 ('시', 2),
 ('지하', 2),
 ('층', 2),
 ('개비', 2),
 ('협소', 1),
 ('헬', 1),
 ('해', 1),
 ('정보', 1),
 ('이여', 1),
 ('애용', 1),
 ('혼잡', 1),
 ('아유', 1),
 ('맘', 1),
 ('내', 1),
 ('살다', 1),
 ('편이',

# 4. 키워드 추출
앞서 Counter로 형태소와 빈도수를 확인하였다. 우리 프로젝트에서 미리 지정해둔 키워드와 대조하여 여부를 체크한다.

다차원 list 처리를 위해 numpy를 이용한다.

In [127]:
import numpy as np

In [129]:
keywords = [["넓다"], ["좁다", "협소"], \
            ["쉽다", "편하다"], ["어렵다", "힘들다"], \
            ["저렴하다", "싸다", "쌈"], ["비싸다"], \
            ["친절하다"], ["불친절하다"], \
            ["좋다", "굿", "굳", "자주", "추천"], ["아쉽다", ""]]

In [130]:
len(keywords)

10

In [131]:
keywords_freq = [0] * 10
# 키워드와 비교하여 Count
frequent = Counter(remove_stopwords).most_common()
for morp in frequent:
    for i in range(len(keywords)):
        for j in range(len(keywords[i])):
            if morp[0] == keywords[i][j]:
                keywords_freq[i] += morp[1]               
    

In [132]:
keywords_freq

[4, 2, 13, 1, 18, 8, 1, 0, 137, 1]

긍정 키워드와 부정 키워드의 비율을 놓고, 
어느 한쪽의 비율이 2:1 보다 클 때만 해당 키워드를 출력한다.

In [142]:
keywords_freq_ratio = np.array(keywords_freq).reshape(5, 2)
keywords_result = np.arange(10).reshape(5, 2)
for i in range(5):
    if keywords_freq_ratio[i][0] + keywords_freq_ratio[i][1] == 0:
        ratio = 1
    elif keywords_freq_ratio[i][0] == 0:
        ratio = 0
    elif keywords_freq_ratio[i][1] == 0:
        ratio = 999
    else:
        ratio = keywords_freq_ratio[i][0] / keywords_freq_ratio[i][1]
    if ratio >= 2:
        keywords_result[i][0] = 1
        keywords_result[i][1] = 0
    elif ratio <= 0.5:
        keywords_result[i][0] = 0
        keywords_result[i][1] = 1
    else:
        keywords_result[i][0] = 0
        keywords_result[i][1] = 0

keywords_result
        

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]])

(5,2) 형태로 어떤 keyword인지 도출되었다.